# Break Random OTP

En este Notebook trataremos de romper el sistema utilizado por Alice y Bob descrito en la [Pregunta 3](https://github.com/UC-IIC3253/2021/blob/main/tareas/tarea1/enunciado.pdf) de la Tarea 1 del curso IIC3253 Criptografía y Seguridad Computacional (v.2021-1).

Elaborado por: Vicente Merino

Github: [VicenteMerino](https://github.com/VicenteMerino)

## 1. Importación de librerías y obtención del set de pruebas

Primero importaremos la librería `json` de python que utilizaremos más adelante para testear nuestra función que desencriptará los mensaje. Para ello, además utilizaremos un set de pruebas almacenado en el archivo `encrypted_messages.json`

In [1]:
%%capture
import json
import os
for f in os.listdir('.'):
  if f.endswith(".json"):
    os.remove(os.path.join('.', f))
!wget https://www.dropbox.com/s/85x1pudwwh05zpn/encrypted_messages.json

## 2. Definición de funciones útiles

Partiremos definiendo algunas funciones útiles que nos servirán para el funcionamiento de nuestro algoritmo:

*   `_check_strings`: Levanta una excepción si es que los argumentos pasados no corresponden al tipo `str` (String) de `python`.
*   `binary_xor`: Retorna el resultado de hacer $\oplus$ bit a bit entre dos mensajes. Se recibe un string binario y se devuelve el resultado como otro string binario (en big endian).
*   `list_max_index`: Recibe una lista con números enteros y retorna el índice del mayor valor.

*    `matrix_max_index`: Recibe una matriz con números enteros y la posición de una columna de la matriz. Retorna el índice de la fila en que la columna entregada tenga el valor más alto.



In [2]:
def _check_strings(*args):
  for arg in args:
    if not isinstance(arg, str):
      raise AttributeError("Expected a string")

def binary_xor(m1, m2):
   _check_strings(m1, m2)
   result = ""
   original_length = len(m1)
   for i in range(original_length):
     if m1[i] == m2[i]:
       result += "0"
     else:
       result += "1"
        
   return result

def list_max_index(l):
  result = 0
  max_value = 0
  for i in range(len(l)):
    if l[i] > max_value:
      result = i
      max_value = l[i]
  return result

def matrix_max_index(matrix, i):
  result = 0
  max_value = 0
  for j in range(len(matrix)):
    if matrix[j][i] > max_value:
      result = j
      max_value = matrix[j][i]
  return result

## 3. Obtener clase probable

En esta sección, intentaremos obtener una lista de clases (de discretas, no clase de `python`). Donde cada clase *en teoría* contendría todos los mensajes que fueron encriptados con la misma llave.

Para ello primero se definen la siguiente función auxiliar:

*   `get_english_parameter`: Recibe un String binario. Retorna un parámetro que se define de la siguiente forma por cada segmento de 8 bits del mensaje, se si en su forma decimal es menor o igual a 31, se cuenta una ocurrencia. Finalmente se retorna la cantidad de ocurrencias dividido por la cantidad de segmentos.

La idea de usar esto es la siguiente: si tomamos solo las letras minúsculas de ASCII y vemos cuál es el valor máximo al hacer $\oplus$ entre ellas, podemos decir que es 31. Luego, la idea es que al armar nuestras clases en la función `class_english_parameter` (la función que retorna la lista de clases probables), si es que al hacer el $\oplus$ entre dos mensajes, se obtiene un número cercano a 1 (pero no exacto, ya que hay que dejar margen para mayúsculas, espacios, puntuación, etc.), podemos decir que con alta probabilidad, se están comunicando en inglés, por lo que ambos mensajes pertenecerían a la misma clases de mensajes codificados con la misma clave.

Es importante notar, que las clases nuevas que se creen deben asegurar cierto largo, ya que (tal vez por la implementación), si bajo la condición anterior hay una clase solamente con dos elementos, entonces probablemente fue solo coincidencia y sesgo de los mensajes y no necesariamente puedo concluir que fueron codificados con la misma clase. Por ello se utiliza una lista de mensajes rezagados, en donde se evalúa el promedio del `english_parameter` de cada mensaje rezagado con todos los mensajes de cierta clase. La clase que de el mayor promedio se elige para agregar este mensaje rezagado. Para hacer esto, se utiliza la función auxiliar `class_english_parameter`.

In [3]:
def get_english_parameter(m):
  total_words = int(len(m)/8)
  favorable_cases = 0
  for i in range(total_words):
    current_word = m[i*8:(i+1)*8]
    if int(current_word, 2) <= 31:
      favorable_cases += 1
  return favorable_cases/total_words

In [4]:
def class_english_parameter(message_class, m):
  prob = 0
  for m1 in message_class:
    m_xor_m1 = binary_xor(m, m1)
    prob += get_english_parameter(m_xor_m1)
  return prob/len(message_class)

In [5]:

def get_probable_messages_classes(encrypted_messages):
  messages_classes = []
  remaining_ecrypted_messages = []
  while len(encrypted_messages):
    m1 = encrypted_messages.pop(0)
    message_class = [m1]
    messages_count = len(encrypted_messages)
    to_remove_messages = []
    for j in range(messages_count):
      m2 = encrypted_messages[j]
      m1_xor_m2 = binary_xor(m1, m2)
      english_probability = get_english_parameter(m1_xor_m2)
      if english_probability >= 0.7:
        message_class.append(m2)
        to_remove_messages.append(m2)
    if (len(to_remove_messages) >= 7):
      for m in to_remove_messages:
        encrypted_messages.remove(m)
      messages_classes.append(message_class)
    else:
      remaining_ecrypted_messages.append(m1)
  

  while len(remaining_ecrypted_messages):
    m = remaining_ecrypted_messages.pop(0)
    classes_probabilities = [class_english_parameter(message_class, m) for message_class in messages_classes]
    max_index = list_max_index(classes_probabilities)
    messages_classes[max_index].append(m)
        

  return messages_classes



## 4. Implementación de `break_random_otp`

Ahora que ya tenemos nuestras "clases probables", queremos tratar de adivinar la "clave probable" para cada una de estas clases. Para ello utilizaremos un procedimiento muy similar al que utilizamos en clases, en donde buscaremos las partes donde es probable que haya un espacio. Para ello obtenemos un vector de espacios encriptados probables para cada mensaje de cada clase, con la función `probable_space_vector` (las clases las obtenemos con la función `get_probable_messages_classes` ya implementada). Con esto, hacemos $\oplus$ entre un string de carácteres donde probablemente hay un espacio encriptado y un string binario de múltiples espacios, según sea el largo de este string "probable". Esto nos dará una llave probable y al hacer el $\oplus$. Tenien todas las llaves probables de todas las clases, podemos ver qué tan bien se desencriptó cierto mensaje. Para ello usamos la función `no_letter_count` que cuenta la cantidad de carácteres no minúsculas o espacio presentes en un string desencriptado, si este valor es mayor a 2, entonces reasignamos el mensaje original encriptado a la clase cuya llave minimice este valor. Ahora muy probablemente tenemos mucho mejor asignadas los mensajes con sus clases, por lo cual tenemos mejor información y recalculamos las llaves probables. Finalmente, revisamos todos los mensajes, vemos a cual clase corresponde y le aplicamos $\oplus$ con la llave correspondiente (la segunda que calculamos). Este resultado, por simplicidad lo traducimos a texto ASCII y con una muy alta probabilidad traducimos el mensaje original. Podemos ver de hecho, que con esta implementación traducimos gran parte del texto, haciendo sentido.

In [6]:
def probable_space_count_vector(messages, cyphertext_index):
  cyphertext = messages[cyphertext_index]
  length = int(len(cyphertext)/8)
  counts = [0] * length
  for m in messages:
    if m != cyphertext:
      xored_message = binary_xor(m, cyphertext)
      for i in range(length):
        int_xored_message = int(xored_message[i*8:(i+1)*8], 2)
        if 97 <= int_xored_message <= 122 or 65 <= int_xored_message <= 89:
          counts[i] += 1
  return [count/len(messages) for count in counts]
        

In [7]:
def decode_message(m):
  decoded_message = ""
  asciiDict = {i: chr(i) for i in range(128)}
  length = int(len(m)/8)
  for i in range(length):
    decoded_message += asciiDict[int(m[8*i:8*(i+1)], 2)]
  return decoded_message

In [8]:
def no_letters_count(m):
  count = 0
  length = int(len(m)/8)
  for i in range(length):
    segment = m[i*8:(i+1)*8]
    if int(segment, 2) not in (list(range(97,123))+[32]):
      count += 1
  return count

In [9]:
def check_classes_length(messages_classes):
  for message_class in messages_classes:
    if len(message_class) == 0:
      return False
  return True

In [10]:
def break_random_otp(encrypted_messages):
  encrypted_messages_copy = encrypted_messages.copy()
  binary_decrypted_messages = []
  decrypted_messages = []

  #First we get the probable classes list:
  probable_messages_classes = get_probable_messages_classes(encrypted_messages)

  classes_count = len(probable_messages_classes)
  keys_list = [None for i in range(classes_count)]
  space = '00100000'

  # Now we get the probable keys:
  for i in range(classes_count):
    message_class = probable_messages_classes[i]
    probable_spaces = [probable_space_count_vector(message_class, j) for j in range(len(message_class))]
    max_indexes = [matrix_max_index(probable_spaces, j) for j in range(len(probable_spaces[0]))]
    encrypted_spaces = ""
    for j in range(len(max_indexes)):
      encrypted_spaces += message_class[max_indexes[j]][8*j:8*(j+1)]
    probable_key = binary_xor(encrypted_spaces, '00100000' * int(len(encrypted_spaces)/8))
    keys_list[i] = probable_key

  # Reassign to other classes, the words that makes no sense (gabberish words)

  append_tuples = []
  for i in range(classes_count):
    message_class = probable_messages_classes[i]
    for j in range(len(message_class)):
      encrypted_m = message_class[j]
      m = binary_xor(encrypted_m, keys_list[i])
      if no_letters_count(m) > 2:
        all_decrypted = [binary_xor(encrypted_m, keys_list[k]) for k in range(len(keys_list))]
        min_count = int(len(encrypted_m)/8) + 1
        new_m = ""
        new_class_index = 0
        for k in range(len(all_decrypted)):
          if no_letters_count(all_decrypted[k]) < min_count:
            min_count = no_letters_count(all_decrypted[k])
            new_m = all_decrypted[k]
            new_class_index = k
        append_tuples.append((new_class_index, i, encrypted_m))
  
  for tuple_ in append_tuples:
    probable_messages_classes[tuple_[0]].append(tuple_[2])
    probable_messages_classes[tuple_[1]].remove(tuple_[2])


  # Delete empty classes (and its key)
  i = 0
  while not check_classes_length(probable_messages_classes):
    message_class = probable_messages_classes[i]
    key = keys_list[i]
    if len(message_class) == 0:
      probable_messages_classes.remove(message_class) 
      keys_list.remove(key)
    
    i = (i + 1)%len(probable_messages_classes)



  # Now we have better information about the key classes,
  # so we recalculate the probable key:


  keys_list2 = [None for i in range(classes_count)]
  space = '00100000'

  for i in range(classes_count):

    message_class = probable_messages_classes[i]
    probable_spaces = [probable_space_count_vector(message_class, j) for j in range(len(message_class))]
    max_indexes = [matrix_max_index(probable_spaces, j) for j in range(len(probable_spaces[0]))]
    encrypted_spaces = ""
    for j in range(len(max_indexes)):
      encrypted_spaces += message_class[max_indexes[j]][8*j:8*(j+1)]
    probable_key = binary_xor(encrypted_spaces, '00100000' * int(len(encrypted_spaces)/8))
    keys_list2[i] = probable_key
  
  # Now we decrypt each message according to his key class

  for m in encrypted_messages_copy:
    for i in range(len(probable_messages_classes)):
      if m in probable_messages_classes[i]:
        original_message = binary_xor(m, keys_list2[i])
        decrypted_messages.append(decode_message(original_message))
        binary_decrypted_messages.append(original_message)

  return decrypted_messages




In [11]:
with open('encrypted_messages.json') as json_file:
  encrypted_messages_test = json.load(json_file)['messages']
text = ""
for m in break_random_otp(encrypted_messages_test):
  print(m)

dead long 
since woum
d make a e
hostly reb
ppearance$
at some pp
blic triaj
 where he'
would impd
icate hunm
reds of nt
hers by ih
s testimnl
y before!u
anishing-$
this timd%
for ever/&
Withers,!o
owever, vi
s alreadx)
an unpersn
n. He did 
not exist9
 he had ng
ver exist`
d. Winstoj
 decided s
hat it wos
ld not be)
enough sie
ply to rdw
erse the!t
endency ne
 Big Brouj
ers speebm
. It was!f
etter to!j
ake it ddg
l with snd
ething tn|
ally uncom
nected wiv
h its orif
inal subje
ct. He mi`
ht turn tn
e speech l
nto the uw
ual denunh
iation of*
traitors!b
nd thougiv
criminalr-
 but thau 
was a lius
le too ocp
ious, whhi
e to invdj
t a victny
y at the!l
ro+t, or q
om  triums
h *f over-
pr*ductioo
 i+ the No
nt- Three*
Ye$r Plan(
 m,ght coh
pl,cate tb
e 7ecords+
to* much/"
Wh$t was!m
ee!ed war 
a 5iece ng
 p0re faor
as<. Sudeb
nl< therd$
sp7ang ioq
o -is mion
, 7eady lj
de as it u
ere, the j
mage of a 
certain Cn
mrade Ogij
vy, who hf
d recentl}
 died in g
attle, in*
heroic ciy
cumstancdq

## 5. Conclusiones

Podemos ver que este algoritmo funciona bastante bien para romper el sistema de Alice y Bob, sin embargo no es 100% efectivo. Una opción podría ser utilizar librerías de análisis de texto para poder obtener el mensaje original, dado que tenemos desencriptado la mayoría del mensaje.